In [1]:
import sqlite3
import sys
import os
import pandas as pd

# 현재 노트북의 디렉터리를 기준으로 루트 디렉터리 경로를 추가
project_root = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('')), ''))
print(project_root)
sys.path.append(project_root)

/Users/jsh/Projects/dacon/ic50-prediction


## Reference
- https://medium.com/standigm/rdkit%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-chembl-%EB%B6%84%EC%9E%90%EB%93%A4-%EC%82%AC%EC%9D%B4%EC%9D%98-%EC%9C%A0%EC%82%AC%EB%8F%84-%EA%B2%80%EC%83%89-60307ccdb441

In [2]:
DATABASE_URL = "../data/chembl_34/chembl_34_sqlite/chembl_34.db"
db = sqlite3.connect(DATABASE_URL)

In [8]:
table_count = [
    (table, count) for table, in db.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
        for count, in db.execute(f"SELECT count(*) FROM {table}").fetchall()
]

table_count_df = pd.DataFrame(table_count, columns=['table', 'count']).sort_values(by='count', ascending=False).reset_index(drop=True)

In [9]:
table_count_df

,table,count
0,activities,20772701
1,activity_properties,10314826
2,chembl_id_lookup,4640251
3,compound_structural_alerts,4436020
4,compound_records,3106257
...,...,...
75,assay_type,6
76,relationship_type,6
77,structural_alert_sets,5
78,curation_lookup,3


In [190]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [36]:
molecule_ids = train_df['Molecule ChEMBL ID'].unique()
molecule_ids_str = ', '.join([f"'{molecule_id}'" for molecule_id in molecule_ids])
print(molecule_ids.shape)

# SQL 쿼리 실행
db.execute(f"SELECT count(*) FROM molecule_dictionary WHERE chembl_id IN ({molecule_ids_str})").fetchall()

(1952,)


[(1952,)]

In [42]:
document_ids = train_df['Document ChEMBL ID'].unique()
document_ids_str = ', '.join([f"'{document_id}'" for document_id in document_ids])
print(document_ids.shape)

db.execute(f"select count(*) from docs where chembl_id in ({document_ids_str})").fetchall()

(66,)


[(66,)]

In [40]:
db.execute("select name from sqlite_master where type='table' and name like '%doc%'").fetchall()

[('docs',)]

In [45]:
document_ids = train_df['Document ChEMBL ID'].unique()
document_ids_str = ', '.join([f"'{document_id}'" for document_id in document_ids])
print(document_ids.shape)

db.execute(f"select count(*) from docs where chembl_id in ({document_ids_str})").fetchall()

(66,)


[(66,)]

In [43]:
db.execute("select name from sqlite_master where type='table' and name like '%assay%'").fetchall()

[('assay_type',),
 ('bioassay_ontology',),
 ('assay_classification',),
 ('assays',),
 ('assay_class_map',),
 ('assay_parameters',)]

In [44]:
document_ids = train_df['Assay ChEMBL ID'].unique()
document_ids_str = ', '.join([f"'{document_id}'" for document_id in document_ids])
print(document_ids.shape)

db.execute(f"select count(*) from assays where chembl_id in ({document_ids_str})").fetchall()

(72,)


[(72,)]

In [52]:
db.execute("select name, type from sqlite_master group by name, type").fetchall()

[('action_type', 'table'),
 ('activities', 'table'),
 ('activity_properties', 'table'),
 ('activity_smid', 'table'),
 ('activity_stds_lookup', 'table'),
 ('activity_supp', 'table'),
 ('activity_supp_map', 'table'),
 ('assay_class_map', 'table'),
 ('assay_classification', 'table'),
 ('assay_classification_pk', 'index'),
 ('assay_parameters', 'table'),
 ('assay_type', 'table'),
 ('assays', 'table'),
 ('atc_classification', 'table'),
 ('binding_sites', 'table'),
 ('bio_component_seqs_pk', 'index'),
 ('bio_component_sequences', 'table'),
 ('bioassay_ontology', 'table'),
 ('biotherapeutic_components', 'table'),
 ('biotherapeutics', 'table'),
 ('bmx_doc_iss', 'index'),
 ('bmx_doc_jrnl', 'index'),
 ('bmx_doc_vol', 'index'),
 ('bmx_doc_year', 'index'),
 ('cell_dictionary', 'table'),
 ('chembl_id_lookup', 'table'),
 ('chembl_release', 'table'),
 ('component_class', 'table'),
 ('component_domains', 'table'),
 ('component_go', 'table'),
 ('component_sequences', 'table'),
 ('component_synonyms', '

In [176]:
doc_ids = ', '.join([f"'{s}'" for s in train_df['Smiles']])
sql = f'''
SELECT m.chembl_id as 'Molecule ChEMBL ID'
     , a.standard_type as 'Standard Type'
     , a.standard_relation as 'Standard Relation'
     , a.standard_value as 'Standard Value'
     , a.standard_units as 'Standard Units'
     , a.pchembl_value as 'pChEMBL Value'
     , aa.chembl_id as 'Assay ChEMBL ID'
     , t.chembl_id as 'Target ChEMBL ID'
     , t.pref_name as 'Target Name'
     , t.organism as 'Target Organism'
     , t.target_type as 'Target Type'
     , d.chembl_id as 'Document ChEMBL ID'
     , a.standard_value as 'IC50_nM'
     , a.pchembl_value as 'pIC50'
     , c.canonical_smiles as 'Smiles'
  FROM compound_structures c
 INNER JOIN molecule_dictionary m ON m.molregno = c.molregno
 INNER JOIN compound_records cr ON cr.molregno = m.molregno
 INNER JOIN activities a ON a.molregno = m.molregno AND a.record_id = cr.record_id AND a.doc_id = cr.doc_id AND a.standard_type = 'IC50' AND a.standard_relation = '=' AND a.standard_units = 'nM'
 INNER JOIN docs d ON d.doc_id = a.doc_id AND d.doc_type IN ('PUBLICATION', 'PATENT')
 INNER JOIN assays aa ON aa.assay_id = a.assay_id AND aa.doc_id = d.doc_id
 INNER JOIN target_dictionary t ON aa.tid = t.tid AND t.target_type = 'SINGLE PROTEIN' AND t.organism = 'Homo sapiens' AND t.chembl_id = 'CHEMBL3778'
 WHERE c.canonical_smiles IN ({doc_ids})
'''
cursor = db.execute(sql)
description = [desc[0] for desc in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns=description)

In [194]:
doc_ids = ', '.join([f"'{s}'" for s in test_df['Smiles']])
sql = f'''
SELECT m.chembl_id as 'Molecule ChEMBL ID'
     , a.standard_type as 'Standard Type'
     , a.standard_relation as 'Standard Relation'
     , a.standard_value as 'Standard Value'
     , a.standard_units as 'Standard Units'
     , a.pchembl_value as 'pChEMBL Value'
     , aa.chembl_id as 'Assay ChEMBL ID'
     , t.chembl_id as 'Target ChEMBL ID'
     , t.pref_name as 'Target Name'
     , t.organism as 'Target Organism'
     , t.target_type as 'Target Type'
     , d.chembl_id as 'Document ChEMBL ID'
     , a.standard_value as 'IC50_nM'
     , a.pchembl_value as 'pIC50'
     , c.canonical_smiles as 'Smiles'
  FROM compound_structures c
 INNER JOIN molecule_dictionary m ON m.molregno = c.molregno
 INNER JOIN compound_records cr ON cr.molregno = m.molregno
 INNER JOIN activities a ON a.molregno = m.molregno AND a.record_id = cr.record_id AND a.doc_id = cr.doc_id AND a.standard_type = 'IC50' AND a.standard_relation = '=' AND a.standard_units = 'nM'
 INNER JOIN docs d ON d.doc_id = a.doc_id AND d.doc_type IN ('PUBLICATION', 'PATENT')
 INNER JOIN assays aa ON aa.assay_id = a.assay_id AND aa.doc_id = d.doc_id
 INNER JOIN target_dictionary t ON aa.tid = t.tid AND t.target_type = 'SINGLE PROTEIN' AND t.organism = 'Homo sapiens' AND t.chembl_id = 'CHEMBL3778'
  WHERE c.canonical_smiles IN ({doc_ids})
'''
cursor = db.execute(sql)
description = [desc[0] for desc in cursor.description]
train_df = pd.DataFrame(cursor.fetchall(), columns=description)

In [196]:
train_df.shape, df.shape

((2415, 15), (2408, 15))

In [204]:
sum(train_df['IC50_nM'].isna()), sum(df['pIC50'].isna())

(0, 0)

In [182]:
test_ddf = pd.read_csv('../data/test.csv')

In [192]:
test_df.shape, test_ddf.shape

((0, 15), (113, 2))

In [193]:
test_df.nunique()

Molecule ChEMBL ID    0
Standard Type         0
Standard Relation     0
Standard Value        0
Standard Units        0
pChEMBL Value         0
Assay ChEMBL ID       0
Target ChEMBL ID      0
Target Name           0
Target Organism       0
Target Type           0
Document ChEMBL ID    0
IC50_nM               0
pIC50                 0
Smiles                0
dtype: int64

In [170]:
df['src_id'].unique()

array([ 1, 37])

In [173]:
df['molregno'].value_counts()

molregno
2318333    6
2519942    6
2204519    5
1966201    5
2329670    5
          ..
2253827    1
2248420    1
2247143    1
2251024    1
2797467    1
Name: count, Length: 1952, dtype: int64

In [174]:
df[df['molregno'].isin([2318333, 2251024])]

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,...,pIC50,Smiles,record_id,molregno,doc_id,compound_key,compound_name,src_id,src_compound_id,cidx
780,CHEMBL4111673,IC50,=,17.50,nM,7.76,CHEMBL3887118,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,7.76,Cn1nccc1Nc1cc(-n2ncc3cc(C#N)cnc32)ncc1C(=O)NC[...,2845578,2251024,102557,BDBM187866,"US9169252, 409",37,None,BDBM187866
1823,CHEMBL4278882,IC50,=,22.00,nM,7.66,CHEMBL4266468,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,7.66,CC(=O)N1CCN([C@H]2CC[C@H](Nc3ncnn4ccc(C(C)C)c3...,3117294,2318333,109756,30,"trans-1-(4-(4-((5-Isopropylpyrrolo[2,1-f][1,2,...",1,None,1094813
1824,CHEMBL4278882,IC50,=,195.00,nM,6.71,CHEMBL4266473,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,6.71,CC(=O)N1CCN([C@H]2CC[C@H](Nc3ncnn4ccc(C(C)C)c3...,3117294,2318333,109756,30,"trans-1-(4-(4-((5-Isopropylpyrrolo[2,1-f][1,2,...",1,None,1094813
1825,CHEMBL4278882,IC50,=,22.00,nM,7.66,CHEMBL4266497,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,7.66,CC(=O)N1CCN([C@H]2CC[C@H](Nc3ncnn4ccc(C(C)C)c3...,3117294,2318333,109756,30,"trans-1-(4-(4-((5-Isopropylpyrrolo[2,1-f][1,2,...",1,None,1094813
1826,CHEMBL4278882,IC50,=,21.88,nM,7.66,CHEMBL4266468,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,7.66,CC(=O)N1CCN([C@H]2CC[C@H](Nc3ncnn4ccc(C(C)C)c3...,3117294,2318333,109756,30,"trans-1-(4-(4-((5-Isopropylpyrrolo[2,1-f][1,2,...",1,None,1094813
1827,CHEMBL4278882,IC50,=,194.98,nM,6.71,CHEMBL4266473,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,6.71,CC(=O)N1CCN([C@H]2CC[C@H](Nc3ncnn4ccc(C(C)C)c3...,3117294,2318333,109756,30,"trans-1-(4-(4-((5-Isopropylpyrrolo[2,1-f][1,2,...",1,None,1094813
2128,CHEMBL4278882,IC50,=,36.00,nM,7.44,CHEMBL4729447,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,7.44,CC(=O)N1CCN([C@H]2CC[C@H](Nc3ncnn4ccc(C(C)C)c3...,3552672,2318333,119278,4,"1-(4-((trans)-4-(5-isopropylpyrrolo[1,2-f][1,2...",1,None,1250041


In [175]:
train_df[train_df['Molecule ChEMBL ID'] == 'CHEMBL4278882']

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,IC50_nM,pIC50,Smiles
1076,CHEMBL4278882,IC50,'=',21.88,nM,7.66,CHEMBL4266468,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4265877,21.88,7.66,CC(=O)N1CCN([C@H]2CC[C@H](Nc3ncnn4ccc(C(C)C)c3...


In [179]:
df.shape, train_df.shape

((2408, 15), (1952, 15))

In [177]:
df.nunique(), train_df.nunique()

(Molecule ChEMBL ID    1952
 Standard Type            1
 Standard Relation        1
 Standard Value         817
 Standard Units           1
 pChEMBL Value          389
 Assay ChEMBL ID        100
 Target ChEMBL ID         1
 Target Name              1
 Target Organism          1
 Target Type              1
 Document ChEMBL ID      69
 IC50_nM                817
 pIC50                  389
 Smiles                1952
 dtype: int64,
 Molecule ChEMBL ID    1952
 Standard Type            1
 Standard Relation        1
 Standard Value         713
 Standard Units           1
 pChEMBL Value          377
 Assay ChEMBL ID         72
 Target ChEMBL ID         1
 Target Name              1
 Target Organism          1
 Target Type              1
 Document ChEMBL ID      66
 IC50_nM                713
 pIC50                  377
 Smiles                1952
 dtype: int64)

In [87]:
doc_ids = ', '.join([f"'{s}'" for s in df['Document ChEMBL ID']])
sql = f'''
SELECT d.*
  FROM docs d
 WHERE d.chembl_id IN ({doc_ids})
'''

cursor = db.execute(sql)
description = [desc[0] for desc in cursor.description]
doc_df = pd.DataFrame(cursor.fetchall(), columns=description)

In [136]:
df.describe()

,Standard Value,pChEMBL Value,IC50_nM,pIC50,doc_id,year,pubmed_id,src_id,chembl_release_id
count,2408.000000,2408.000000,2408.000000,2408.000000,2408.000000,2408.000000,1.200000e+03,2408.000000,2408.000000
mean,665.804917,7.425257,665.804917,7.425257,102677.378738,2016.193106,2.921698e+07,19.059801,24.406146
std,2552.973831,1.100465,2552.973831,1.100465,16980.964769,3.020165,4.902917e+06,18.003639,6.082619
min,0.022000,4.260000,0.022000,4.260000,31167.000000,2006.000000,1.656375e+07,1.000000,1.000000
25%,4.875000,6.540000,4.875000,6.540000,102557.000000,2015.000000,2.628870e+07,1.000000,24.000000
50%,22.000000,7.660000,22.000000,7.660000,102877.000000,2015.000000,2.917250e+07,37.000000,24.000000
75%,286.250000,8.312500,286.250000,8.312500,109756.000000,2017.000000,3.309185e+07,37.000000,26.000000
max,55000.000000,10.660000,55000.000000,10.660000,126114.000000,2023.000000,3.720145e+07,37.000000,34.000000


In [137]:
train_df.describe()

,Standard Value,pChEMBL Value,IC50_nM,pIC50
count,1952.000000,1952.000000,1952.000000,1952.000000
mean,649.001365,7.518586,649.001365,7.518586
std,2639.946734,1.107959,2639.946734,1.107959
min,0.022000,4.260000,0.022000,4.260000
25%,4.100000,6.680000,4.100000,6.680000
50%,15.250000,7.820000,15.250000,7.820000
75%,209.197500,8.390000,209.197500,8.390000
max,55000.000000,10.660000,55000.000000,10.660000


In [143]:
standard_value_diff = set(df['pChEMBL Value'].unique()) - set(train_df['pChEMBL Value'].unique())
print(len(standard_value_diff))
ddf = df[df['pChEMBL Value'].isin(standard_value_diff)]

12


In [159]:
df['patent_id'].isna()

0       True
1       True
2       True
3       True
4       True
        ... 
2403    True
2404    True
2405    True
2406    True
2407    True
Name: patent_id, Length: 2408, dtype: bool

In [149]:
ddf[ddf['journal'].isna() == False]

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,...,chembl_id,title,doc_type,authors,abstract,patent_id,ridx,src_id,chembl_release_id,contact
311,CHEMBL3622518,IC50,=,1900.00,nM,5.72,CHEMBL3625606,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL3621203,Discovery and Structure Enabled Synthesis of 2...,PUBLICATION,"Seganish WM, Fischmann TO, Sherborne B, Matasi...",We report the identification and synthesis of ...,None,CLD0,1,22,None
1615,CHEMBL4075552,IC50,=,115.00,nM,6.94,CHEMBL4015947,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4014327,"Discovery of Clinical Candidate 1-{[(2S,3S,4S)...",PUBLICATION,"Lee KL, Ambler CM, Anderson DR, Boscoe BP, Bre...",Through fragment-based drug design focused on ...,None,28498658,1,25,None
1618,CHEMBL4071526,IC50,=,29.50,nM,7.53,CHEMBL4015947,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4014327,"Discovery of Clinical Candidate 1-{[(2S,3S,4S)...",PUBLICATION,"Lee KL, Ambler CM, Anderson DR, Boscoe BP, Bre...",Through fragment-based drug design focused on ...,None,28498658,1,25,None
1664,CHEMBL4105582,IC50,=,23000.00,nM,4.64,CHEMBL4026983,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4024763,Discovery and Optimization of Pyrrolopyrimidin...,PUBLICATION,"Scott JS, Degorce SL, Anjum R, Culshaw J, Davi...",Herein we report the optimization of a series ...,None,29172502,1,25,None
1812,CHEMBL4283453,IC50,=,2340.00,nM,5.63,CHEMBL4266473,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4265877,Optimization of permeability in a series of py...,PUBLICATION,"Degorce SL, Anjum R, Dillman KS, Drew L, Groom...",We have developed a series of orally efficacio...,None,29398441,1,26,None
1814,CHEMBL4283453,IC50,=,2344.23,nM,5.63,CHEMBL4266473,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4265877,Optimization of permeability in a series of py...,PUBLICATION,"Degorce SL, Anjum R, Dillman KS, Drew L, Groom...",We have developed a series of orally efficacio...,None,29398441,1,26,None
1834,CHEMBL4282348,IC50,=,1650.00,nM,5.78,CHEMBL4266473,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4265877,Optimization of permeability in a series of py...,PUBLICATION,"Degorce SL, Anjum R, Dillman KS, Drew L, Groom...",We have developed a series of orally efficacio...,None,29398441,1,26,None
1836,CHEMBL4282348,IC50,=,1659.59,nM,5.78,CHEMBL4266473,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4265877,Optimization of permeability in a series of py...,PUBLICATION,"Degorce SL, Anjum R, Dillman KS, Drew L, Groom...",We have developed a series of orally efficacio...,None,29398441,1,26,None
1839,CHEMBL4293607,IC50,=,21.38,nM,7.67,CHEMBL4266468,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4265877,Optimization of permeability in a series of py...,PUBLICATION,"Degorce SL, Anjum R, Dillman KS, Drew L, Groom...",We have developed a series of orally efficacio...,None,29398441,1,26,None
2068,CHEMBL4783351,IC50,=,1375.00,nM,5.86,CHEMBL4673036,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,...,CHEMBL4665913,"Discovery of CA-4948, an Orally Bioavailable I...",PUBLICATION,"Gummadi VR,Boruah A,Ainan BR,Vare BR,Manda S,G...",Small molecule potent IRAK4 inhibitors from a ...,None,33335659,1,30,None


In [99]:
doc_df.nunique()

doc_id               69
journal               6
year                 17
volume               31
issue                27
first_page           64
last_page            64
pubmed_id            64
doi                  64
chembl_id            69
title                69
doc_type              2
authors              62
abstract             63
patent_id             5
ridx                 43
src_id                2
chembl_release_id    19
contact               0
dtype: int64

In [121]:
assay_ids = ', '.join([f"'{s}'" for s in df['Assay ChEMBL ID']])
sql = f'''
SELECT d.*
  FROM assays d
 WHERE d.chembl_id IN ({assay_ids})
'''

cursor = db.execute(sql)
description = [desc[0] for desc in cursor.description]
assay_df = pd.DataFrame(cursor.fetchall(), columns=description)

assay_ids = ', '.join([f"'{s}'" for s in train_df['Assay ChEMBL ID']])
cursor = db.execute(sql)
description = [desc[0] for desc in cursor.description]
assay_train_df = pd.DataFrame(cursor.fetchall(), columns=description)

In [122]:
assay_df.nunique(), assay_train_df.nunique()

(assay_id                      100
 doc_id                         69
 description                    74
 assay_type                      1
 assay_test_type                 0
 assay_category                  0
 assay_organism                  1
 assay_tax_id                    1
 assay_strain                    0
 assay_tissue                    1
 assay_cell_type                 6
 assay_subcellular_fraction      0
 tid                             1
 relationship_type               2
 confidence_score                2
 curated_by                      3
 src_id                          2
 src_assay_id                   34
 chembl_id                     100
 cell_id                         3
 bao_format                      4
 tissue_id                       1
 variant_id                      0
 aidx                           64
 dtype: int64,
 assay_id                      100
 doc_id                         69
 description                    74
 assay_type                      1
 assa

In [78]:
doc_ids = ', '.join([f"'{s}'" for s in train_df['Document ChEMBL ID']])
sql = f'''
SELECT d.*
  FROM docs d
 WHERE d.chembl_id IN ({doc_ids})
'''

cursor = db.execute(sql)
description = [desc[0] for desc in cursor.description]
doc_train_df = pd.DataFrame(cursor.fetchall(), columns=description)

In [100]:
doc_train_df.nunique()

doc_id               66
journal               6
year                 17
volume               29
issue                27
first_page           61
last_page            61
pubmed_id            61
doi                  61
chembl_id            66
title                66
doc_type              2
authors              59
abstract             60
patent_id             5
ridx                 41
src_id                2
chembl_release_id    19
contact               0
dtype: int64

In [92]:
set(doc_df['doc_id'].unique()) - set(doc_train_df['doc_id'].unique())

{np.int64(47102), np.int64(107047), np.int64(119003)}

In [98]:
doc_df[doc_df['doc_id'].isin([47102, 107047, 119003])]

,doc_id,journal,year,volume,issue,first_page,last_page,pubmed_id,doi,chembl_id,title,doc_type,authors,abstract,patent_id,ridx,src_id,chembl_release_id,contact
0,47102,J Med Chem,2009,52,10,3191,3204,19397322.0,10.1021/jm800861c,CHEMBL1138507,"Synthesis, activity, and pharmacophore develop...",PUBLICATION,"Hall MD, Salam NK, Hellawell JL, Fales HM, Ken...",We have recently identified a new class of com...,None,CLD0,1,2,None
37,107047,Eur J Med Chem,2017,141,None,657,675,29107425.0,10.1016/j.ejmech.2017.10.003,CHEMBL4145442,Novel LCK/FMS inhibitors based on phenoxypyrim...,PUBLICATION,"Farag AK, Elkamhawy A, Londhe AM, Lee KT, Pae ...",Tyrosine kinases including LCK and FMS are inv...,None,29107425,1,26,None
52,119003,Bioorg Med Chem,2018,26,8.0,1740,1750,29523467.0,10.1016/j.bmc.2018.02.022,CHEMBL4715818,Novel quinazoline derivatives bearing various ...,PUBLICATION,"Hou W,Ren Y,Zhang Z,Sun H,Ma Y,Yan B",A series of novel quinazoline derivatives bear...,None,29523467,1,30,None


In [113]:
doc_df[doc_df['doc_id'].isin([47102, 107047, 119003]) == False]

,doc_id,journal,year,volume,issue,first_page,last_page,pubmed_id,doi,chembl_id,title,doc_type,authors,abstract,patent_id,ridx,src_id,chembl_release_id,contact
1,38711,Bioorg Med Chem Lett,2008,18,11,3211,3214,18474425.0,10.1016/j.bmcl.2008.04.058,CHEMBL1143094,IRAK-4 inhibitors. Part 1: a series of amides.,PUBLICATION,"Buckley GM, Gowers L, Higueruelo AP, Jenkins K...",The synthesis and profile of a series of amide...,None,CLD0,1,1,None
2,39491,Bioorg Med Chem Lett,2008,18,12,3656,3660,18501603.0,10.1016/j.bmcl.2008.04.042,CHEMBL1145577,IRAK-4 inhibitors. Part III: a series of imida...,PUBLICATION,"Buckley GM, Fosbeary R, Fraser JL, Gowers L, H...",Following the identification of a potent IRAK ...,None,CLD0,1,1,None
3,31167,Bioorg Med Chem Lett,2006,16,11,2842,2845,16563752.0,10.1016/j.bmcl.2006.03.020,CHEMBL1147720,Discovery and initial SAR of inhibitors of int...,PUBLICATION,"Powers JP, Li S, Jaen JC, Liu J, Walker NP, Wa...",High-throughput screening of a small-molecule ...,None,CLD0,1,1,None
4,42165,Bioorg Med Chem Lett,2008,18,11,3291,3295,18482836.0,10.1016/j.bmcl.2008.04.039,CHEMBL1150395,IRAK-4 inhibitors. Part II: a structure-based ...,PUBLICATION,"Buckley GM, Ceska TA, Fraser JL, Gowers L, Gro...",A potent IRAK-4 inhibitor was identified throu...,None,CLD0,1,2,None
5,52613,Leukemia,2009,23,3,477,485,19039322.0,10.1038/leu.2008.334,CHEMBL1240341,Global target profile of the kinase inhibitor ...,PUBLICATION,"Remsing Rix LL, Rix U, Colinge J, Hantschel O,...",The detailed molecular mechanism of action of ...,None,CLD0,1,10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,124270,J Med Chem,2021,64,3.0,1283,1345,33481605.0,10.1021/acs.jmedchem.0c01511,CHEMBL5149965,Small-Molecule Kinase Inhibitors for the Treat...,PUBLICATION,"Xie Z, Yang X, Duan Y, Han J, Liao C.",Great successes have been achieved in developi...,None,33481605,1,33,None
65,124282,J Med Chem,2021,64,6.0,2878,2900,33719439.0,10.1021/acs.jmedchem.0c01851,CHEMBL5149977,FLT3 Inhibitors in Acute Myeloid Leukemia: Cha...,PUBLICATION,"Wang Z, Cai J, Cheng J, Yang W, Zhu Y, Li H, L...",Mutations in the FMS-like tyrosine kinase 3 (<...,None,33719439,1,33,None
66,125655,Bioorg Med Chem,2020,28,23.0,115815,115815,33091850.0,10.1016/j.bmc.2020.115815,CHEMBL5244269,Improving metabolic stability and removing ald...,PUBLICATION,"Degorce SL, Aagaard A, Anjum R, Cumming IA, Di...","In this article, we report our efforts towards...",None,33091850,1,34,None
67,126082,Bioorg Med Chem,2023,87,None,117302,117302,37201454.0,10.1016/j.bmc.2023.117302,CHEMBL5257151,"Design, synthesis, and pharmacological evaluat...",PUBLICATION,"Inami H, Mizutani T, Watanabe J, Hayashida H, ...",Interleukin-1 receptor-associated kinase 4 (IR...,None,37201454,1,34,None


In [110]:
doc_df[doc_df['contact'].isna()]

,doc_id,journal,year,volume,issue,first_page,last_page,pubmed_id,doi,chembl_id,title,doc_type,authors,abstract,patent_id,ridx,src_id,chembl_release_id,contact
0,47102,J Med Chem,2009,52,10,3191,3204,19397322.0,10.1021/jm800861c,CHEMBL1138507,"Synthesis, activity, and pharmacophore develop...",PUBLICATION,"Hall MD, Salam NK, Hellawell JL, Fales HM, Ken...",We have recently identified a new class of com...,None,CLD0,1,2,None
1,38711,Bioorg Med Chem Lett,2008,18,11,3211,3214,18474425.0,10.1016/j.bmcl.2008.04.058,CHEMBL1143094,IRAK-4 inhibitors. Part 1: a series of amides.,PUBLICATION,"Buckley GM, Gowers L, Higueruelo AP, Jenkins K...",The synthesis and profile of a series of amide...,None,CLD0,1,1,None
2,39491,Bioorg Med Chem Lett,2008,18,12,3656,3660,18501603.0,10.1016/j.bmcl.2008.04.042,CHEMBL1145577,IRAK-4 inhibitors. Part III: a series of imida...,PUBLICATION,"Buckley GM, Fosbeary R, Fraser JL, Gowers L, H...",Following the identification of a potent IRAK ...,None,CLD0,1,1,None
3,31167,Bioorg Med Chem Lett,2006,16,11,2842,2845,16563752.0,10.1016/j.bmcl.2006.03.020,CHEMBL1147720,Discovery and initial SAR of inhibitors of int...,PUBLICATION,"Powers JP, Li S, Jaen JC, Liu J, Walker NP, Wa...",High-throughput screening of a small-molecule ...,None,CLD0,1,1,None
4,42165,Bioorg Med Chem Lett,2008,18,11,3291,3295,18482836.0,10.1016/j.bmcl.2008.04.039,CHEMBL1150395,IRAK-4 inhibitors. Part II: a structure-based ...,PUBLICATION,"Buckley GM, Ceska TA, Fraser JL, Gowers L, Gro...",A potent IRAK-4 inhibitor was identified throu...,None,CLD0,1,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,124270,J Med Chem,2021,64,3.0,1283,1345,33481605.0,10.1021/acs.jmedchem.0c01511,CHEMBL5149965,Small-Molecule Kinase Inhibitors for the Treat...,PUBLICATION,"Xie Z, Yang X, Duan Y, Han J, Liao C.",Great successes have been achieved in developi...,None,33481605,1,33,None
65,124282,J Med Chem,2021,64,6.0,2878,2900,33719439.0,10.1021/acs.jmedchem.0c01851,CHEMBL5149977,FLT3 Inhibitors in Acute Myeloid Leukemia: Cha...,PUBLICATION,"Wang Z, Cai J, Cheng J, Yang W, Zhu Y, Li H, L...",Mutations in the FMS-like tyrosine kinase 3 (<...,None,33719439,1,33,None
66,125655,Bioorg Med Chem,2020,28,23.0,115815,115815,33091850.0,10.1016/j.bmc.2020.115815,CHEMBL5244269,Improving metabolic stability and removing ald...,PUBLICATION,"Degorce SL, Aagaard A, Anjum R, Cumming IA, Di...","In this article, we report our efforts towards...",None,33091850,1,34,None
67,126082,Bioorg Med Chem,2023,87,None,117302,117302,37201454.0,10.1016/j.bmc.2023.117302,CHEMBL5257151,"Design, synthesis, and pharmacological evaluat...",PUBLICATION,"Inami H, Mizutani T, Watanabe J, Hayashida H, ...",Interleukin-1 receptor-associated kinase 4 (IR...,None,37201454,1,34,None


In [111]:
doc_train_df[doc_train_df['contact'].isna()]

,doc_id,journal,year,volume,issue,first_page,last_page,pubmed_id,doi,chembl_id,title,doc_type,authors,abstract,patent_id,ridx,src_id,chembl_release_id,contact
0,38711,Bioorg Med Chem Lett,2008,18,11,3211,3214,18474425.0,10.1016/j.bmcl.2008.04.058,CHEMBL1143094,IRAK-4 inhibitors. Part 1: a series of amides.,PUBLICATION,"Buckley GM, Gowers L, Higueruelo AP, Jenkins K...",The synthesis and profile of a series of amide...,None,CLD0,1,1,None
1,39491,Bioorg Med Chem Lett,2008,18,12,3656,3660,18501603.0,10.1016/j.bmcl.2008.04.042,CHEMBL1145577,IRAK-4 inhibitors. Part III: a series of imida...,PUBLICATION,"Buckley GM, Fosbeary R, Fraser JL, Gowers L, H...",Following the identification of a potent IRAK ...,None,CLD0,1,1,None
2,31167,Bioorg Med Chem Lett,2006,16,11,2842,2845,16563752.0,10.1016/j.bmcl.2006.03.020,CHEMBL1147720,Discovery and initial SAR of inhibitors of int...,PUBLICATION,"Powers JP, Li S, Jaen JC, Liu J, Walker NP, Wa...",High-throughput screening of a small-molecule ...,None,CLD0,1,1,None
3,42165,Bioorg Med Chem Lett,2008,18,11,3291,3295,18482836.0,10.1016/j.bmcl.2008.04.039,CHEMBL1150395,IRAK-4 inhibitors. Part II: a structure-based ...,PUBLICATION,"Buckley GM, Ceska TA, Fraser JL, Gowers L, Gro...",A potent IRAK-4 inhibitor was identified throu...,None,CLD0,1,2,None
4,52613,Leukemia,2009,23,3,477,485,19039322.0,10.1038/leu.2008.334,CHEMBL1240341,Global target profile of the kinase inhibitor ...,PUBLICATION,"Remsing Rix LL, Rix U, Colinge J, Hantschel O,...",The detailed molecular mechanism of action of ...,None,CLD0,1,10,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,124270,J Med Chem,2021,64,3.0,1283,1345,33481605.0,10.1021/acs.jmedchem.0c01511,CHEMBL5149965,Small-Molecule Kinase Inhibitors for the Treat...,PUBLICATION,"Xie Z, Yang X, Duan Y, Han J, Liao C.",Great successes have been achieved in developi...,None,33481605,1,33,None
62,124282,J Med Chem,2021,64,6.0,2878,2900,33719439.0,10.1021/acs.jmedchem.0c01851,CHEMBL5149977,FLT3 Inhibitors in Acute Myeloid Leukemia: Cha...,PUBLICATION,"Wang Z, Cai J, Cheng J, Yang W, Zhu Y, Li H, L...",Mutations in the FMS-like tyrosine kinase 3 (<...,None,33719439,1,33,None
63,125655,Bioorg Med Chem,2020,28,23.0,115815,115815,33091850.0,10.1016/j.bmc.2020.115815,CHEMBL5244269,Improving metabolic stability and removing ald...,PUBLICATION,"Degorce SL, Aagaard A, Anjum R, Cumming IA, Di...","In this article, we report our efforts towards...",None,33091850,1,34,None
64,126082,Bioorg Med Chem,2023,87,None,117302,117302,37201454.0,10.1016/j.bmc.2023.117302,CHEMBL5257151,"Design, synthesis, and pharmacological evaluat...",PUBLICATION,"Inami H, Mizutani T, Watanabe J, Hayashida H, ...",Interleukin-1 receptor-associated kinase 4 (IR...,None,37201454,1,34,None


In [83]:
df.nunique()

Molecule ChEMBL ID    1952
Standard Type            1
Standard Relation        1
Standard Value         817
Standard Units           1
pChEMBL Value          389
Assay ChEMBL ID        100
Target ChEMBL ID         1
Target Name              1
Target Organism          1
Target Type              1
Document ChEMBL ID      69
IC50_nM                817
pIC50                  389
Smiles                1952
doc_id                  69
journal                  6
year                    17
volume                  31
issue                   27
first_page              64
last_page               64
pubmed_id               64
doi                     64
chembl_id               69
title                   69
doc_type                 2
authors                 62
abstract                63
patent_id                5
ridx                    43
src_id                   2
chembl_release_id       19
contact                  0
dtype: int64

In [84]:
train_df.nunique()

Molecule ChEMBL ID    1952
Standard Type            1
Standard Relation        1
Standard Value         713
Standard Units           1
pChEMBL Value          377
Assay ChEMBL ID         72
Target ChEMBL ID         1
Target Name              1
Target Organism          1
Target Type              1
Document ChEMBL ID      66
IC50_nM                713
pIC50                  377
Smiles                1952
dtype: int64

In [60]:
train_df['Target ChEMBL ID']

0       CHEMBL3778
1       CHEMBL3778
2       CHEMBL3778
3       CHEMBL3778
4       CHEMBL3778
           ...    
1947    CHEMBL3778
1948    CHEMBL3778
1949    CHEMBL3778
1950    CHEMBL3778
1951    CHEMBL3778
Name: Target ChEMBL ID, Length: 1952, dtype: object

In [ ]:
while True:
    sql = input()
    if sql == 'd':
        break
    cursor = db.execute(sql)
    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    print(df)
    cursor.close()

---
# test.csv 의 smiles는 chembl에 등록되지 않은 대상
- 따라서 train set에서 얻을 수 있는 feature 정보들을 직접 활용이 불가함
- 우회할 수 있는 방법으로, 각 test smiles와 임계 유사도 이상을 갖는 molecule를 기준으로 train 과 동일한 feature들을 추출해 통계값을 test smiles의 feature로 대체하고자 함


In [236]:
# ChEBML API 로 임계 유사도 이상 molecule 정보 조회하기
import requests
import xml.etree.ElementTree as ET

# SMILES 문자열
test_df = pd.read_csv('../data/test.csv')
similarity_threshold = 50  # 유사도 임계값 (0-100 사이)
output_df = {}
for smiles in test_df['Smiles']: # 예시로 Ethanol 사용
    # ChEMBL API URL
    url = f"https://www.ebi.ac.uk/chembl/api/data/similarity/{smiles}/{similarity_threshold}"

    # 요청 보내기
    response = requests.get(url)

    # 결과 확인
    if response.status_code == 200:
        # XML 파싱
        root = ET.fromstring(response.text)

        # molecule_chembl_id 추출
        chembl_ids = [elem.text for elem in root.findall(".//molecule_chembl_id")]

        chembl_ids = ', '.join([f"'{s}'" for s in chembl_ids])

        print(chembl_ids)

        sql = f'''
        SELECT m.chembl_id as 'Molecule ChEMBL ID'
            , a.standard_type as 'Standard Type'
            , a.standard_relation as 'Standard Relation'
            , a.standard_value as 'Standard Value'
            , a.standard_units as 'Standard Units'
            , a.pchembl_value as 'pChEMBL Value'
            , aa.chembl_id as 'Assay ChEMBL ID'
            , t.chembl_id as 'Target ChEMBL ID'
            , t.pref_name as 'Target Name'
            , t.organism as 'Target Organism'
            , t.target_type as 'Target Type'
            , d.chembl_id as 'Document ChEMBL ID'
            , a.standard_value as 'IC50_nM'
            , a.pchembl_value as 'pIC50'
            , c.canonical_smiles as 'Smiles'
        FROM compound_structures c
        INNER JOIN molecule_dictionary m ON m.molregno = c.molregno
        INNER JOIN compound_records cr ON cr.molregno = m.molregno
        INNER JOIN activities a ON a.molregno = m.molregno AND a.record_id = cr.record_id AND a.doc_id = cr.doc_id AND a.standard_type = 'IC50' AND a.standard_relation = '=' AND a.standard_units = 'nM'
        INNER JOIN docs d ON d.doc_id = a.doc_id AND d.doc_type IN ('PUBLICATION', 'PATENT')
        INNER JOIN assays aa ON aa.assay_id = a.assay_id AND aa.doc_id = d.doc_id
        INNER JOIN target_dictionary t ON aa.tid = t.tid AND t.target_type = 'SINGLE PROTEIN' AND t.organism = 'Homo sapiens' AND t.chembl_id = 'CHEMBL3778'
        WHERE m.chembl_id IN ({chembl_ids})
        '''
        cursor = db.execute(sql)
        description = [desc[0] for desc in cursor.description]
        test = pd.DataFrame(cursor.fetchall(), columns=description)

        output_df[smiles] = test
    else:
        print(f"Error: {response.status_code}")


output_df = pd.DataFrame(index=output_df.keys(), data=output_df.values())

'CHEMBL257127', 'CHEMBL257127', 'CHEMBL254546', 'CHEMBL254546', 'CHEMBL404514', 'CHEMBL404514', 'CHEMBL403662', 'CHEMBL403662', 'CHEMBL259797', 'CHEMBL259797', 'CHEMBL407821', 'CHEMBL407821', 'CHEMBL3354514', 'CHEMBL3354514'
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404

'CHEMBL3676302', 'CHEMBL3676302', 'CHEMBL3676304', 'CHEMBL3676304', 'CHEMBL3676303', 'CHEMBL3676303', 'CHEMBL3676258', 'CHEMBL3676258', 'CHEMBL3676268', 'CHEMBL3676268'


Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404

Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Error: 404



Error: 404

Error: 404
Error: 404
Error: 404
Error: 404
Error: 404

Error: 404
Error: 404
Error: 404

Error: 404
Error: 404
Error: 404
Error: 404
Error: 404
Er

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (22,) + inhomogeneous part.